In [1]:
from __future__ import division
import random
import turtle
from shapely.geometry import Point, LineString
import math
import copy


# Function to convert binary to decimal points
def binarytoDecimal(val):
    return int(val, 2)


# Returning a binary string with fixed size for decimal input
def decimaltoBinary(val):
    return '{0:08b}'.format(val)


# Generating a population of coordinates with size n
def generate_population(size, n):
    new_population = []
    for i in range(size):
        # Generating chromosomes
        chromosome = []
        # n coordinates
        for j in range(n):
            random_x = random.randint(0, 255)
            random_y = random.randint(0, 255)
            coordinates = [decimaltoBinary(random_x), decimaltoBinary(random_y)]
            chromosome.append(coordinates)
        new_population.append(chromosome)
    return new_population


# using turtle library of python to draw the polygon based on its coordinates
def drawPolygon(chromosome, n):
    window = turtle.Screen()
    draw = turtle.Turtle()

    # Setting initial coordinates to draw polygon
    draw.penup()
    x_coord = binarytoDecimal(chromosome[0][0])
    y_coord = binarytoDecimal(chromosome[0][1])
    draw.goto(x_coord, y_coord)
    draw.pendown()

    # Drawing the remaining four coordinates of the polygon
    for i in range(n - 1):
        x_coord = binarytoDecimal(chromosome[i + 1][0])
        y_coord = binarytoDecimal(chromosome[i + 1][1])
        draw.goto(x_coord, y_coord)

    x_coord = binarytoDecimal(chromosome[0][0])
    y_coord = binarytoDecimal(chromosome[0][1])
    draw.goto(x_coord, y_coord)

    window.mainloop()


# using LineString from shapely library to determine the intersection points
def intersection_points(chromosome, n):
    coordinates = []
    for i in range(n):
        decimalValues = [binarytoDecimal(chromosome[i][0]), binarytoDecimal(chromosome[i][1])]
        coordinates.append(Point(decimalValues))

    # LineString generates a complete line between two coordinates, which allows us to detect intersections
    lines = []
    for i in range(n - 1):
        lines.append(LineString([coordinates[i], (coordinates[i + 1])]))

    lines.append(LineString([coordinates[n - 1], (coordinates[0])]))

    # List to store intersecting points of the polygon
    intersections = []
    for i in range(n):
        for j in range(i + 1, n):
            intersecting_point = lines[i].intersection(lines[j])
            if intersecting_point:
                # Checking if intersection is besides the joining point of both polygons
                if intersecting_point not in coordinates:
                    intersections.append(intersecting_point)

    return intersections


# Using tan, sin and cos to determine the angle between two vectors
def test_angle(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    dot = x1 * x2 + y1 * y2
    det = x1 * y2 - y1 * x2
    angle = math.atan2(det, dot)
    ans = math.degrees(angle)
    if ans < 0:
        ans = -1 * ans
        ans = 360 - ans
    return ans


# Function used to determine whether the polygon contains any concave angles or not (which are not convex)
def convex_check(chromosome, n):
    coordinates = []
    for i in range(n):
        decimalValues = [binarytoDecimal(chromosome[i][0]), binarytoDecimal(chromosome[i][1])]
        coordinates.append(decimalValues)

    # Making vectors and finding angles from it
    angles = []
    for i in range(n):
        vector_points = []
        iterator = i
        if i + 1 > n - 1:
            x_axis = coordinates[0][0] - coordinates[iterator][0]
            y_axis = coordinates[0][1] - coordinates[iterator][1]
            vector_points.append([x_axis, y_axis])

        else:
            x_axis = coordinates[iterator + 1][0] - coordinates[iterator][0]
            y_axis = coordinates[iterator + 1][1] - coordinates[iterator][1]
            vector_points.append([x_axis, y_axis])

        if i - 1 < 0:
            x_axis = coordinates[n - 1][0] - coordinates[iterator][0]
            y_axis = coordinates[n - 1][1] - coordinates[iterator][1]
            vector_points.append([x_axis, y_axis])

        else:
            x_axis = coordinates[iterator - 1][0] - coordinates[iterator][0]
            y_axis = coordinates[iterator - 1][1] - coordinates[iterator][1]
            vector_points.append([x_axis, y_axis])

        ang = test_angle(vector_points[0], vector_points[1])
        angles.append(ang)

    number_of_convex = 0
    for ang in angles:
        if ang > 180:
            number_of_convex += ang

    return number_of_convex


# Performing crossover on the input chromosomes
def crossover(chromosome1, chromosome2, polygon_sides):
    new_chromosome1 = []
    new_chromosome2 = []
    for i in range(polygon_sides):
        chromosome1_x = copy.deepcopy(chromosome1[i][0])
        chromosome1_y = copy.deepcopy(chromosome1[i][1])

        chromosome2_x = copy.deepcopy(chromosome2[i][0])
        chromosome2_y = copy.deepcopy(chromosome2[i][1])

        new_chromosome1_x = ""
        new_chromosome1_y = ""
        new_chromosome2_x = ""
        new_chromosome2_y = ""

        for j in range(8):
            if j < 8 / 2:
                new_chromosome1_x += chromosome2_x[j]
                new_chromosome1_y += chromosome2_y[j]

                new_chromosome2_x += chromosome1_x[j]
                new_chromosome2_y += chromosome1_y[j]
            else:
                new_chromosome2_x += chromosome2_x[j]
                new_chromosome2_y += chromosome2_y[j]

                new_chromosome1_x += chromosome1_x[j]
                new_chromosome1_y += chromosome1_y[j]

        new_chromosome1.append([new_chromosome1_x, new_chromosome1_y])
        new_chromosome2.append([new_chromosome2_x, new_chromosome2_y])

    return new_chromosome1, new_chromosome2


def mutation(chromosome, polygon_size):
    new_chromosome = []
    for i in range(polygon_size):
        index_x = random.randint(0, 7)  # Changing value at random index chromosome in population
        index_y = random.randint(0, 7)
        x_coordinate = copy.deepcopy(chromosome[i][0])
        y_coordinate = copy.deepcopy(chromosome[i][1])

        new_x_coordinate = ""
        new_y_coordinate = ""

        for j in range(8):
            if j == index_x:
                if x_coordinate[index_x] == "1":
                    new_x_coordinate += "0"
                else:
                    new_x_coordinate += "1"

            else:
                new_x_coordinate += x_coordinate[j]

            if j == index_y:
                if y_coordinate[index_y] == "1":
                    new_y_coordinate += "0"
                else:
                    new_y_coordinate += "1"
            else:
                new_y_coordinate += y_coordinate[j]

        new_chromosome.append([new_x_coordinate, new_y_coordinate])

    return new_chromosome


# Downsizing the population based on general
def roulette_wheel(input_population, initial_size, polygon_size):
    # First calculating the total fitness of the population
    sum_fitness = 0
    for i in range(len(input_population)):
        sum_fitness += fitness_function(input_population[i], polygon_size)

    # Calculating probability of each chromosome's fitness
    chromosome_probability = []
    for i in range(len(input_population)):
        chromosome_probability.append(fitness_function(input_population[i], polygon_size) / sum_fitness)

    # Shifting the probabilities, because this is a minimization problem
    chromosome_probability = [1 - val for val in chromosome_probability]

    # Choosing population based on their probability
    new_population = [val for _, val in sorted(zip(chromosome_probability, input_population))]
    new_population = copy.deepcopy(random.sample(new_population, initial_size))

    return new_population


def fitness_function(chromosome, n):
    continuous_fitness = 0

    # First testing for intersection
    intersects = intersection_points(chromosome, n)

    # Logic: The nearer the intersection is to its nearest points, the more chances are that the next gen will have
    # better fitness
    if len(intersects) != 0:
        coordinates = []
        for i in range(n):
            decimalValues = [binarytoDecimal(chromosome[i][0]), binarytoDecimal(chromosome[i][1])]
            coordinates.append(Point(decimalValues))

        # Summing the nearest distances and adding them to
        for j in range(len(intersects)):
            sum_top_n = 0
            distances = []
            for i in range(n):
                point_distance = intersects[j].distance(coordinates[i])
                distances.append(point_distance)

            # Sorting in descending order
            distances.sort(reverse=True)
            for i in range(n // 2):
                sum_top_n += distances[i]

            continuous_fitness += sum_top_n

    # Now testing for convex angles
    convex_angle = convex_check(chromosome, n)
    if convex_angle != 0:
        continuous_fitness += convex_angle

    return continuous_fitness


# The main genetic algorithm function that implements GA and produces the fittest polygon
def genetic_algorithm(sides, size):
    poly_sides = sides
    population_size = size
    population = generate_population(population_size, poly_sides)

    # Minimum value from the fitness function will be considered the fittest polygon
    # This is a minimization problem
    highest_fitness = float('inf')  # Initializing with infinity
    fittest_Polygon = None
    ga_Iterations = 0
    while True:
        if ga_Iterations > 1000:
            print(f"{ga_Iterations} iterations reached, the best polygon found has the fitness value {highest_fitness}")
            break

        print(f"\n Genetic Algorithm Iterations = {ga_Iterations}")
        for x in range(population_size):
            fitness_Value = fitness_function(population[x], poly_sides)
            if fitness_Value <= highest_fitness:
                highest_fitness = fitness_Value
                fittest_Polygon = population[x]

        if highest_fitness != 0:
            print(f"Highest fitness in this iteration = {highest_fitness}")
            # Performing crossover
            for x in range(population_size // 2):
                child1, child2 = crossover(population[x], population[x + 1], poly_sides)
                population.append(child1)
                population.append(child2)

            # Performing mutation
            for x in range(population_size // 2):
                mutated_child = mutation(population[x], poly_sides)
                population.append(mutated_child)

            # Performing selection using roulette wheel
            population = copy.deepcopy(roulette_wheel(population, population_size, poly_sides))

        else:
            print("Fitness of zero reached (Minimization), best polygon found")
            break
        ga_Iterations += 1

    # Printing the fittest polygon using turtle library
    drawPolygon(fittest_Polygon, poly_sides)



In [2]:
while True:
    n_sides = int(input("Enter the number of sides of polygon (3-15): "))
    if 2 < n_sides <= 15:
        break
    print("Invalid sides entered, please enter between the limit")

genetic_algorithm(n_sides, 100)

Enter the number of sides of polygon (3-15): 6

 Genetic Algorithm Iterations = 0
Highest fitness in this iteration = 213.93896161421705

 Genetic Algorithm Iterations = 1
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 2
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 3
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 4
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 5
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 6
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 7
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 8
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 9
Highest fitness in this iteration = 200.8074087303246

 Genetic Algorithm Iterations = 10
Highest fitness in this it